# **Lab1: Regression**
In *lab 1*, you need to finish:

1.  Basic Part: Implement the regression model to predict people's grip force from their weight.
You can use either Matrix Inversion or Gradient Descent.


> *   Step 1: Split Data
> *   Step 2: Preprocess Data
> *   Step 3: Implement Regression
> *   Step 4: Make Prediction
> *   Step 5: Train Model and Generate Result

2.  Advanced Part: Implementing a regression model to predict grip force in a different way (for example, with more variables) than the basic part




---
# 1. Basic Part (50%)
In the first part, you need to implement the regression to predict grip force

Please save the prediction result in a CSV file and submit it to Kaggle

### Import Packages

> Note: You **cannot** import any other package


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

### Global attributes
Define the global attributes\
You can also add your own global attributes here

In [2]:
training_dataroot = 'lab1_basic_training.csv' # Training data file file named as 'lab1_basic_training.csv'
testing_dataroot = 'lab1_basic_testing.csv'   # Testing data file named as 'lab1_basic_testing.csv'
output_dataroot = 'lab1_basic.csv' # Output file will be named as 'lab1_basic.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be a list with 100 elements

### Load the Input File
First, load the basic input file **lab1_basic_training.csv** and **lab1_basic_testing.csv**

Input data would be stored in *training_datalist* and *testing_datalist*

In [3]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = pd.read_csv(training_dataroot).to_numpy()

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = pd.read_csv(testing_dataroot).to_numpy()

### Implement the Regression Model

> Note: It is recommended to use the functions we defined, you can also define your own functions

#### Step 1: Split Data
Split data in *training_datalist* into training dataset and validation dataset


In [4]:
def SplitData(data, split_ratio):
    """
    Splits the given dataset into training and validation sets based on the specified split ratio.

    Parameters:
    - data (numpy.ndarray): The dataset to be split. It is expected to be a 2D array where each row represents a data point and each column represents a feature.
    - split_ratio (float): The ratio of the data to be used for training. For example, a value of 0.8 means 80% of the data will be used for training and the remaining 20% for validation.

    Returns:
    - training_data (numpy.ndarray): The portion of the dataset used for training.
    - validation_data (numpy.ndarray): The portion of the dataset used for validation.

    """
    mid = int(data.shape[0] * split_ratio)
    training_data = data[:mid]
    validation_data = data[mid:]

    # TODO

    return training_data, validation_data



#### Step 2: Preprocess Data
Handle unreasonable data and missing data

> Hint 1: Outliers and missing data can be addressed by either removing them or replacing them using statistical methods (e.g., the mean of all data).

> Hint 2: Missing data are represented as `np.nan`, so functions like `np.isnan()` can be used to detect them.

> Hint 3: Methods such as the Interquartile Range (IQR) can help detect outliers

In [5]:
def PreprocessData(data):
    """
    Preprocess the given dataset and return the result.

    Parameters:
    - data (numpy.ndarray): The dataset to preprocess. It is expected to be a 2D array where each row represents a data point and each column represents a feature.

    Returns:
    - preprocessedData (numpy.ndarray): Preprocessed data.
    """
    data = data[~np.isnan(data[:]).any(axis=1)]
    q1 = np.percentile(data[:,0], 25)
    q3 = np.percentile(data[:,0], 75)
    
    iqr = q3-q1
    lowerbound = q1 - (1.5 * iqr)
    upperbound = q3 + (1.5 * iqr)
    
    detect = (data[:, 0] >= lowerbound) & (data[:,0] <= upperbound)
    preprocessedData = data[detect]

    # TODO


    return preprocessedData


### Step 3: Implement Regression
You have to use Gradient Descent to finish this part

In [6]:
def Regression(dataset):
    """
    Performs regression on the given dataset and return the coefficients.

    Parameters:
    - dataset (numpy.ndarray): A 2D array where each row represents a data point.

    Returns:
    - w (numpy.ndarray): The coefficients of the regression model. For example, y = w[0] + w[1] * x + w[2] * x^2 + ...
    """
    
    X = dataset[:, :1]
    y = dataset[:, 1]
    # TODO: Decide on the degree of the polynomial
    degree = 2  # For example, quadratic regression

    # Add polynomial features to X
    X_poly = np.ones((X.shape[0], 1))  # Add intercept term (column of ones)
    for d in range(1, degree + 1):
        X_poly = np.hstack((X_poly, X ** d))  # Add x^d terms to feature matrix

    # Initialize coefficients (weights) to zero
    num_dimensions = X_poly.shape[1]  # Number of features (including intercept and polynomial terms)
    w = np.zeros(num_dimensions)

    # TODO: Set hyperparameters
    num_iteration = 50000
    learning_rate = 0.000000041


    # Gradient Descent
    m = len(y)  # Number of data points

    for iteration in range(num_iteration):
        # TODO: Prediction using current weights and compute error
        y_predict = np.dot(X_poly, w)
        #print(y_predict)
        err = y_predict - y
        #print(err)

        # TODO: Compute gradient
        gradient = 2/m * np.dot(X_poly.T, err)
        #print(gradient)

        # TODO: Update the weights
        w -= (learning_rate * gradient)
        #print(w)
        
        # TODO: Optionally, print the cost every 100 iterations
        if iteration % 100 == 0:
            cost = np.sum(err**2) / m
            print(f"Iteration {iteration}, Cost: {cost}")

    return w


### Step 4: Make Prediction
Make prediction of testing dataset and store the value in *output_datalist*

In [9]:

def MakePrediction(w, test_dataset):
    """
    Predicts the output for a given test dataset using a regression model.

    Parameters:
    - w (numpy.ndarray): The coefficients of the model, where each element corresponds to
                               a coefficient for the respective power of the independent variable.
    - test_dataset (numpy.ndarray): A 1D array containing the input values (independent variable)
                                          for which predictions are to be made.

    Returns:
    - list/numpy.ndarray: A list or 1d array of predicted values corresponding to each input value in the test dataset.
    """
    prediction = []

    # TODO
    degree = 2
    X_poly = np.ones((test_dataset[:,:1].shape[0], 1))
    for d in range(1, degree+1):
        X_poly = np.hstack((X_poly, test_dataset[:, :1]**d))
    print(X_poly)
    print(w)
    prediction = np.dot(X_poly, w)
    print(prediction)
    return prediction


### Step 5: Train Model and Generate Result

Use the above functions to train your model on training dataset, and predict the answer of testing dataset.

Save your predicted values in `output_datalist`

> Notice: **Remember to inclue the coefficients of your model in the report**



In [10]:
# TODO
# (1) Split data
train, val = SplitData(training_datalist, 0.95)

# (2) Preprocess data
preptrain = PreprocessData(train)
prepval = PreprocessData(val)

# (3) Train regression model
grad = Regression(preptrain)
print(grad)

# (4) Predict validation dataset's answer, calculate MAPE comparing to the ground truth
pred = MakePrediction(grad, prepval)
gt = prepval[:, 1]
mape = np.mean(np.abs((gt - pred)/gt))*100
print(mape)

# (5) Make prediction of testing dataset and store the values in output_datalist
output_datalist = MakePrediction(grad, testing_datalist)

Iteration 0, Cost: 3164.3499778833498
Iteration 100, Cost: 740.6930983177439
Iteration 200, Cost: 719.8664926448713
Iteration 300, Cost: 719.5503181981751
Iteration 400, Cost: 719.4094932709212
Iteration 500, Cost: 719.2704404785906
Iteration 600, Cost: 719.1316776448226
Iteration 700, Cost: 718.9931915700789
Iteration 800, Cost: 718.8549815948521
Iteration 900, Cost: 718.7170471677645
Iteration 1000, Cost: 718.5793877394499
Iteration 1100, Cost: 718.4420027616447
Iteration 1200, Cost: 718.3048916871788
Iteration 1300, Cost: 718.168053969973
Iteration 1400, Cost: 718.0314890650365
Iteration 1500, Cost: 717.8951964284649
Iteration 1600, Cost: 717.7591755174392
Iteration 1700, Cost: 717.6234257902213
Iteration 1800, Cost: 717.487946706154
Iteration 1900, Cost: 717.3527377256576
Iteration 2000, Cost: 717.2177983102285
Iteration 2100, Cost: 717.0831279224363
Iteration 2200, Cost: 716.9487260259227
Iteration 2300, Cost: 716.8145920853982
Iteration 2400, Cost: 716.6807255666404
Iteration 250

Iteration 21800, Cost: 695.1688585715445
Iteration 21900, Cost: 695.0781469715125
Iteration 22000, Cost: 694.9876162224018
Iteration 22100, Cost: 694.897265963651
Iteration 22200, Cost: 694.807095835418
Iteration 22300, Cost: 694.7171054785774
Iteration 22400, Cost: 694.6272945347202
Iteration 22500, Cost: 694.537662646152
Iteration 22600, Cost: 694.448209455891
Iteration 22700, Cost: 694.358934607668
Iteration 22800, Cost: 694.2698377459234
Iteration 22900, Cost: 694.1809185158065
Iteration 23000, Cost: 694.0921765631747
Iteration 23100, Cost: 694.0036115345907
Iteration 23200, Cost: 693.9152230773221
Iteration 23300, Cost: 693.82701083934
Iteration 23400, Cost: 693.7389744693168
Iteration 23500, Cost: 693.651113616626
Iteration 23600, Cost: 693.5634279313396
Iteration 23700, Cost: 693.4759170642275
Iteration 23800, Cost: 693.3885806667558
Iteration 23900, Cost: 693.3014183910857
Iteration 24000, Cost: 693.2144298900714
Iteration 24100, Cost: 693.1276148172593
Iteration 24200, Cost: 6

Iteration 42900, Cost: 679.5321900430093
Iteration 43000, Cost: 679.4726531233249
Iteration 43100, Cost: 679.4132349017989
Iteration 43200, Cost: 679.3539351417836
Iteration 43300, Cost: 679.294753607103
Iteration 43400, Cost: 679.2356900620521
Iteration 43500, Cost: 679.1767442713955
Iteration 43600, Cost: 679.1179160003675
Iteration 43700, Cost: 679.0592050146693
Iteration 43800, Cost: 679.0006110804703
Iteration 43900, Cost: 678.9421339644057
Iteration 44000, Cost: 678.8837734335757
Iteration 44100, Cost: 678.8255292555451
Iteration 44200, Cost: 678.7674011983422
Iteration 44300, Cost: 678.7093890304576
Iteration 44400, Cost: 678.6514925208434
Iteration 44500, Cost: 678.5937114389125
Iteration 44600, Cost: 678.5360455545375
Iteration 44700, Cost: 678.4784946380496
Iteration 44800, Cost: 678.4210584602383
Iteration 44900, Cost: 678.3637367923496
Iteration 45000, Cost: 678.3065294060858
Iteration 45100, Cost: 678.2494360736048
Iteration 45200, Cost: 678.1924565675178
Iteration 45300, 

### *Write the Output File*

Write the prediction to output csv and upload the file to Kaggle
> Format: 'Id', 'gripForce'


In [19]:
# Assume that output_datalist is a list (or 1d array) with length = 100

with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Id', 'gripForce'])
  for i in range(len(output_datalist)):
    writer.writerow([i,output_datalist[i]])


# 2. Advanced Part (45%)
In the second part, you need to implement regression differently from the basic part to improve your grip force predictions. You must use more than two features.

You can choose either matrix inversion or gradient descent for this part

We have provided `lab1_advanced_training.csv` for your training

> Notice: Be cautious of the "gender" attribute, as it is represented by "F"/"M" rather than a numerical value.

Please save the prediction result in a CSV file and submit it to Kaggle

In [16]:
training_dataroot = 'lab1_advanced_training.csv' # Training data file file named as 'lab1_advanced_training.csv'
testing_dataroot = 'lab1_advanced_testing.csv'   # Testing data file named as 'lab1_advanced_testing.csv'
output_dataroot = 'lab1_advanced.csv' # Output file will be named as 'lab1_advanced.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be a list with 3000 elements

In [17]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = pd.read_csv(training_dataroot).to_numpy()

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = pd.read_csv(testing_dataroot).to_numpy()

In [37]:
def SplitData(data, split_ratio):
    """
    Splits the given dataset into training and validation sets based on the specified split ratio.

    Parameters:
    - data (numpy.ndarray): The dataset to be split. It is expected to be a 2D array where each row represents a data point and each column represents a feature.
    - split_ratio (float): The ratio of the data to be used for training. For example, a value of 0.8 means 80% of the data will be used for training and the remaining 20% for validation.

    Returns:
    - training_data (numpy.ndarray): The portion of the dataset used for training.
    - validation_data (numpy.ndarray): The portion of the dataset used for validation.

    """
    np.random.seed(42)
    shuffle_idx = np.random.permutation(data.shape[0])
    data = data[shuffle_idx]
    mid = int(data.shape[0] * split_ratio)
    training_data = data[:mid]
    validation_data = data[mid:]

    # TODO

    return training_data, validation_data



In [38]:
def PreprocessData(data):
    """
    Preprocess the given dataset and return the result.

    Parameters:
    - data (numpy.ndarray): The dataset to preprocess. It is expected to be a 2D array where each row represents a data point and each column represents a feature.

    Returns:
    - preprocessedData (numpy.ndarray): Preprocessed data.
    """
    for i in range(data.shape[0]):
        if data[i][1] == 'F':
            data[i][1] = 0
        elif data[i][1] == 'M':
            data[i][1] = 1
    data = np.array(data, dtype = float)
    data = data[data[:, 7] > 0]
    data = data[data[:, 7] < 150]
    
    preprocessedData = data
    for i in range(8):
        if i != 1:
            q1 = np.nanpercentile(data[:,i], 25)
            q3 = np.nanpercentile(data[:,i], 75)

            iqr = q3-q1
            lowerbound = q1 - (1.5 * iqr)
            upperbound = q3 + (1.5 * iqr)

            detect = (preprocessedData[:, i] >= lowerbound) & (preprocessedData[:,i] <= upperbound)
            preprocessedData = preprocessedData[detect]
    
    avg = np.nanmean(preprocessedData, axis = 0)
    uniqueval, cnt = np.unique(preprocessedData[:, 1][~np.isnan(preprocessedData[:, 1])], return_counts = True)
    avg[1] = uniqueval[np.argmax(cnt)]
    for i in range(preprocessedData.shape[1]):
        preprocessedData[np.isnan(preprocessedData[:, i]), i] = avg[i]
        #TRY REMOVE OUTLIER FIRST BEFORE REPLACING NAN WITH AVG
    

    # TODO
    low = np.min(preprocessedData, axis = 0)
    high = np.max(preprocessedData, axis = 0)
    preprocessedData = (preprocessedData - low)/(high-low)

    return preprocessedData, low, high

def PreprocessDataTest(data, low, high):
    for i in range(data.shape[0]):
        if data[i][1] == 'F':
            data[i][1] = 0
        elif data[i][1] == 'M':
            data[i][1] = 1
    if data.shape[1] == 8:
        data = data[data[:, 7] > 0]
        data = data[data[:, 7] < 150]
    data = np.array(data, dtype = float)
    avg = []
    for i in range(data.shape[1]):
        if i != 1:
            feat = data[:, i]
            q1 = np.nanpercentile(feat, 25)
            q3 = np.nanpercentile(feat, 75)
            
            iqr = q3 - q1
            lowerbound = q1 - (1.5 * iqr)
            upperbound = q3 + (1.5 * iqr)
            print(lowerbound, upperbound)
            print(feat[(feat >=lowerbound) & (feat <=upperbound)])
            avg.append(np.mean(feat[(feat >= lowerbound) & (feat <= upperbound)]))
            
            data[:, i] = np.where((feat < lowerbound) | (feat > upperbound), avg[i], feat)
        else:
            avg.append(0)
    
    uniqueval, cnt = np.unique(data[:, 1][~np.isnan(data[:, 1])], return_counts = True)
    avg[1] = uniqueval[np.argmax(cnt)]
    print(avg)
    print(data)
    for i in range(data.shape[1]):
        data[np.isnan(data[:, i]), i] = avg[i]
    preprocessedData = (data-low)/(high-low)
    
    return preprocessedData

In [39]:
def Regression(dataset):
    """
    Performs regression on the given dataset and return the coefficients.

    Parameters:
    - dataset (numpy.ndarray): A 2D array where each row represents a data point.

    Returns:
    - w (numpy.ndarray): The coefficients of the regression model. For example, y = w[0] + w[1] * x + w[2] * x^2 + ...
    """
    
    X = dataset[:, :7]
    y = dataset[:, 7]
    
    # TODO: Decide on the degree of the polynomial
    degree = 3  # For example, quadratic regression

    # Add polynomial features to X
    X_poly = np.ones((X.shape[0], 1))  # Add intercept term (column of ones)
    for d in range(1, degree + 1):
        X_poly = np.hstack((X_poly, X**d)) # Add x^d terms to feature matrix
    #for i in range(X.shape[1]-1):
     #   for j in range(i+1, X.shape[1]):
      #      X_poly = np.hstack((X_poly, np.array([X[:, i] * X[:, j]]).T))
        
    # Initialize coefficients (weights) to zero
    num_dimensions = X_poly.shape[1]  # Number of features (including intercept and polynomial terms)
    w = np.zeros(num_dimensions)

    # TODO: Set hyperparameters
    num_iteration = 10000
    learning_rate = 0.01


    # Gradient Descent
    m = len(y)  # Number of data points
    lambdaval = 0.1
    for iteration in range(num_iteration):
        # TODO: Prediction using current weights and compute error
        y_predict = np.dot(X_poly, w)
        #print(y_predict)
        err = y_predict - y
        #print(err)

        # TODO: Compute gradient
        gradient = 2/m * np.dot(X_poly.T, err) + 2 *lambdaval*w
        #print(gradient)

        # TODO: Update the weights
        w -= (learning_rate * gradient)
        #print(w)
        
        # TODO: Optionally, print the cost every 100 iterations
        if iteration % 100 == 0:
            cost = np.sum(err**2) / m + lambdaval * np.sum(w**2)
            print(f"Iteration {iteration}, Cost: {cost}")

    return w


In [40]:

def MakePrediction(w, test_dataset):
    """
    Predicts the output for a given test dataset using a regression model.

    Parameters:
    - w (numpy.ndarray): The coefficients of the model, where each element corresponds to
                               a coefficient for the respective power of the independent variable.
    - test_dataset (numpy.ndarray): A 1D array containing the input values (independent variable)
                                          for which predictions are to be made.

    Returns:
    - list/numpy.ndarray: A list or 1d array of predicted values corresponding to each input value in the test dataset.
    """
    prediction = []

    X = test_dataset[:, :7]
    # TODO
    degree = 3
    X_poly = np.ones((X.shape[0], 1))
    for d in range(1, degree+1):
        X_poly = np.hstack((X_poly, X**d))
    #for i in range(X.shape[1]-1):
     #   for j in range(i+1, X.shape[1]):
      #      X_poly = np.hstack((X_poly, np.array([X[:, i] * X[:, j]]).T))
    
    prediction = np.dot(X_poly, w)

    return prediction


In [41]:
# TODO
# (1) Split data
train, val = SplitData(training_datalist, 0.6)

# (2) Preprocess data
preptrain, a, b = PreprocessData(train)
prepval = PreprocessDataTest(val, a, b)
preptest = PreprocessDataTest(testing_datalist, a[:7], b[:7])

# (3) Train regression model
grad = Regression(preptrain)
print(f'GRAD:', grad)
# (4) Predict validation dataset's answer, calculate MAPE comparing to the ground truth
pred = MakePrediction(grad, prepval)
print(pred)
gt = prepval[:, 7]
pred = pred * (b[7] - a[7]) + a[7]
gt = gt * (b[7] - a[7]) + a[7]
print(gt)
print(pred)
print(np.abs((gt-pred)))
mape = np.mean(np.abs((gt - pred)/gt))*100
print(f'MAPE:', mape)

# (5) Make prediction of testing dataset and store the values in output_datalist
output_datalist = MakePrediction(grad, preptest)
output_datalist = output_datalist * (b[7] - a[7]) + a[7]

-9.5 82.5
[43. 21. 22. ... 36. 46. 61.]
143.85000000000002 193.04999999999995
[170.  177.2 184.6 ... 156.2 162.3 158.4]
30.45 102.85000000000001
[73.8 64.2 90.6 ... 63.5 61.2 54.7]
3.0 43.0
[22.7 15.3 23.3 ... 34.3 21.7 27.8]
48.5 108.5
[86. 78. 64. ... 83. 78. 73.]
88.5 172.5
[126. 139. 125. ... 108. 123. 145.]
0.30000000000000426 87.5
[50.9 43.1 57.2 ... 32.2 37.6 27.7]
[36.97740846533368, 1.0, 168.49683937823835, 67.06708160442601, 23.048717430171962, 78.90252341311134, 130.4124513618677, 44.261895959862684]
[[ 43.    1.  170.  ...  86.  126.   50.9]
 [ 21.    1.  177.2 ...  78.  139.   43.1]
 [ 22.    1.  184.6 ...  64.  125.   57.2]
 ...
 [ 36.    0.  156.2 ...  83.  108.   32.2]
 [ 46.    0.  162.3 ...  78.  123.   37.6]
 [ 61.    0.  158.4 ...  73.  145.   27.7]]
-8.0 80.0
[39. 27. 36. ... 21. 27. 55.]
145.45000000000002 192.24999999999997
[180.1 174.3 168.6 ... 168.4 164.1 169.6]
32.5625 101.4625
[73.8 79.6 59.1 ... 66.7 52.3 67.3]
3.599999999999998 42.0
[28.1 20.5 31.4 ... 22.

In [36]:
# Assume that output_datalist is a list (or 1d array) with length = 100

with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Id', 'gripForce'])
  for i in range(len(output_datalist)):
    writer.writerow([i,output_datalist[i]])


# Save the Code File
Please save your code and submit it as an ipynb file! (**Lab1.ipynb**)